In [ ]:
import numpy as np
from array import array
import math
from tqdm import tqdm

In [ ]:
data = np.load('flame_1201x1201x1201.npy')

In [ ]:
# Padding from to 1201x1201x1201 to 1202x1202x1202
dataPadding = np.zeros((1202, 1202, 1202), dtype=np.float32)
dataPadding[:1201, :1201, :1201] = data

# Padding to Cube

In [ ]:
dataPadding[1201, :1201, :1201] = data[1200, :, :]
dataPadding[:1201, 1201, :1201] = data[:, 1200, :]
dataPadding[:1201, :1201, 1201] = data[:, :, 1200]
dataPadding[1201, 1201, 1201] = dataPadding[1201, 1201, 1200]

## Partition
|             | Level 1 (far)    | Level 2          | Level 3         | Level 4         |
| ----------- | -----------      | -----------      | -----------     | -----------     |
| x           | 2x2x2            | 4x4x4            | 8x8x8           | 16x16x16        |

In [ ]:
# Partition the input volume into micro-blocks with ghost area (one extra sample on the boundary) for each level of detail
def partitionVolume(path, data, xSize, ySize, zSize, xBlockNum, yBlockNum, zBlockNum, xBlockSize, yBlockSize, zBlockSize, blockIndexShift):
    print("Input volume size:", data.shape) # 1201x1201x1201
    xSkip = int(16/xBlockNum) # 8/4/2/16
    ySkip = int(16/yBlockNum) # 8/4
    zSkip = int(16/zBlockNum) # 8/4
    dataNew = data[0:xSize:xSkip, 0:ySize:ySkip, 0:zSize:zSkip] # 151x151x151 / 301x301x301
    xSize, ySize, zSize = dataNew.shape
    print(xSize, ySize, zSize)
    dataNewNew = np.zeros((xSize + 1, ySize + 1, zSize + 1)) # 152x152x152
    dataNewNew[0:xSize, 0:ySize, 0:zSize] = dataNew
    dataNewNew[xSize, :, :] = dataNewNew[xSize - 1, :, :]
    dataNewNew[:, ySize, :] = dataNewNew[:, ySize - 1, :]
    dataNewNew[:, :, zSize] = dataNewNew[:, :, zSize - 1]
    print("Down-sampled volume size:", dataNew.shape)
    print("intermiddle volume size:", dataNewNew.shape)
    print("Down-sampled max: ", np.max(dataNew))
    
    xstep = xBlockSize - 2 # 77 - 2 = 75
    ystep = yBlockSize - 2 # 77 - 2 = 75
    zstep = zBlockSize - 2 # 77 - 2 = 75
    
    blocks = []
    for x in tqdm(range(zBlockNum)): # 2/4
        for y in range(yBlockNum): # 2/4
            for z in range(xBlockNum): # 2/4
                xStart = x*xstep           # 0   # 75
                xEnd = xStart + xstep + 2  # 77  # 152
                yStart = y*ystep
                yEnd = yStart + ystep + 2
                zStart = z*zstep
                zEnd = zStart + zstep + 2
                block = dataNewNew[xStart:xEnd, yStart:yEnd, zStart:zEnd]
                blocks.append(block)

    # save blocks to storage
    for i in tqdm(range(len(blocks))):
        fileName = path + "/" + str(i + blockIndexShift) + ".blk"
        f = open(fileName, "wb")
        dataList = blocks[i].flatten().tolist()
        array("f", dataList).tofile(f)
    print("Done...")
    return blocks

In [ ]:
blocks = partitionVolume("data_blocks_flame", data, 1201, 1201, 1201, 2, 2, 2, 77, 77, 77, 0)

In [ ]:
blocks = partitionVolume("data_blocks_flame", data, 1201, 1201, 1201, 4, 4, 4, 77, 77, 77, 8)

In [ ]:
blocks = partitionVolume("data_blocks_flame", data, 1201, 1201, 1201, 8, 8, 8, 77, 77, 77, 8 + 64)

In [ ]:
blocks = partitionVolume("data_blocks_flame", data, 1201, 1201, 1201, 16, 16, 16, 77, 77, 77, 8 + 64 + 512)